In [1]:
#what
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences 

In [ ]:
xdf = pd.read_csv('/content/drive/MyDrive/ML/Datasets/Chinese_segmentation/as_training.utf8', header=None, names=['raw_words'])
xfor_vocab = open('/content/drive/MyDrive/ML/Datasets/Chinese_segmentation/as_training.utf8')

#testdf = pd.read_csv('/content/drive/MyDrive/ML/Datasets/Chinese_segmentation/as_test.utf8', header=None)

In [7]:
xdf = pd.read_csv(r'C:\Users\zhiha\OneDrive\Documents\.Zhihao\.ML\datasets\Chinese Segmentation\as_training.utf8', header=None, names=['raw_words'])
#xfor_vocab = open('/content/drive/MyDrive/ML/Datasets/Chinese_segmentation/as_training.utf8')

In [8]:
def spaceornot(line):
  splace = []
  listed = list(line)
  for i, char in enumerate(listed):
    if listed[i] == listed[-1]:
      splace.append(1)
    elif char == '\u3000':
      pass
    elif listed[i + 1] == '\u3000':
      splace.append(1)
    elif listed[i + 1] != '\u3000':
      splace.append(0)
  return splace

In [9]:
yraw = xdf.copy()

xtrain = pd.DataFrame(xdf['raw_words'].str.replace('\u3000',''))   #could ignore last punctuation

#tokening _input data
tokenizer = Tokenizer(char_level=True, oov_token='<OOV>')
        #dlist_sent = seriestolist(xtrain, 0)
tokenizer.fit_on_texts(xtrain['raw_words'])
word_index = tokenizer.word_index
xtrain['sequenced_sent'] = tokenizer.texts_to_sequences(xtrain['raw_words'])
X = pad_sequences(xtrain['sequenced_sent'], padding='post')

#y_actual
yraw['labels'] = yraw['raw_words'].map(spaceornot)
Y = pad_sequences(yraw['labels'], padding='post')


# alternatively, you can pass the padded arrays into the dataframe by converting it into a list using list()
"""
used data:
train_data: xseq
train_label: yactual
"""

'\nused data:\ntrain_data: xseq\ntrain_label: yactual\n'

In [10]:
val_split = 0.75
vsize = len(word_index)
sentlen = len(X[0,:])
em_dim = 512
epol = 1000
basize = 512

#callbacks

In [11]:
#splitting dataset
spli = int(val_split * len(X))
trainX, testX = X[:spli], X[spli:]
trainY, testY = Y[:spli], Y[spli:]


#possible to split in ~model~.fit() function too

In [12]:
'''
Custom Loss
'''

class MaskedSequenceLoss(tf.keras.losses.Loss):
    def __init__(
        self,
        average_across_timesteps=False,
        average_across_batch=False,
        sum_over_timesteps=True,
        sum_over_batch=True,
        softmax_loss_function=None,
        name=None,
        reduction=None, # dummy arg so it can be used as custom object when loading saved model
    ):
        super().__init__()
        self.opts = {
            "average_across_timesteps": average_across_timesteps,
            "average_across_batch": average_across_batch,
            "sum_over_timesteps": sum_over_timesteps,
            "sum_over_batch": sum_over_batch,
            "softmax_loss_function": softmax_loss_function,
            "name": name,
        }
    
    def call(self, y_true, y_pred):
        return tfa.seq2seq.sequence_loss(y_pred, y_true,
                                         weights=tf.cast(y_pred._keras_mask, tf.float32) if hasattr(y_pred, "_keras_mask") else tf.ones(y_true.shape),
                                         **self.opts)

In [ ]:
em_dim_eff = 128


xin = Input((sentlen,))

x = Embedding(vsize, em_dim_eff, input_length=sentlen, mask_zero=True)(xin)
x = Bidirectional(LSTM(64, return_sequences = True))(x)

x = Dense(64, activation='swish')(x)
x =Dropout(0.5)(x)

xout = Dense(2, activation='linear')(x)

effsegcl = Model(xin, xout)
effsegcl.compile(optimizer='adam', loss=MaskedSequenceLoss(), metrics=['accuracy'])
effsegcl.summary()

In [42]:
xin = Input((sentlen,))

x = Embedding(vsize, em_dim, input_length=sentlen, mask_zero = True)(xin) #ignore padded zeros -> mask_zero = True
x = Bidirectional(LSTM(64, return_sequences=True))(x)
x = Bidirectional(LSTM(64, return_sequences=True))(x)

x = Dense(256, activation='swish')(x)
x = Dropout(0.7)(x)
x = Dense(64, activation='swish')(x)
x = Dropout(0.7)(x)

xout = Dense(2, activation='linear')(x)

segcl = Model(xin, xout)
segcl.compile(optimizer='adam', loss=MaskedSequenceLoss(), metrics=['accuracy'])
segcl.summary()

Model: "model_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_19 (InputLayer)       [(None, 188)]             0         
                                                                 
 embedding_7 (Embedding)     (None, 188, 512)          3115520   
                                                                 
 bidirectional_15 (Bidirecti  (None, 188, 128)         295424    
 onal)                                                           
                                                                 
 bidirectional_16 (Bidirecti  (None, 188, 128)         98816     
 onal)                                                           
                                                                 
 dense_75 (Dense)            (None, 188, 256)          33024     
                                                                 
 dropout_14 (Dropout)        (None, 188, 256)          0  

In [ ]:
xin = Input((sentlen, 1))
x = tf.expand_dims(xin, axis=1)
x = Flatten()(x)

x = Dense(1024, activation='swish')(xin)
x = Dense(512, activation='swish')(x)
x = Dense(256, activation='swish')(x)
x = Dense(256, activation='swish')(x)

xout1 = Dense(sentlen)(x)
xout2 = Dense(sentlen)(x)

xout = tf.stack([xout1, xout2], axis=2)

FULL = Model(xin, xout)
FULL.compile(optimizer='adam', loss=MaskedSequenceLoss(), metrics=['accuracy'])
FULL.summary()

In [58]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', patience=10, factor=0.1, verbose=1)
]

In [ ]:
FULL.fit(
    trainX, 
    trainY,
    validation_data=(testX, testY),
    epochs = epol,
    batch_size = basize,
    callbacks = callbacks
)

#you can use validation_split = True to split the data into val and train, instead of manual splitting
#using tf.stack, you can pass the dataframe series into it directly
'''
segcl.fit(

)
'''


In [15]:
def splitter(sentences, text_split_indexes):
    dat = pd.DataFrame(sentences, columns=['unsplit'])
    splited = []
    for i, sentence in enumerate(sentences):
        split_sen = ''
        for n, sply in enumerate(list(text_split_indexes[i])[:len(sentence)]):
            split_sen += sentence[n]
            if sply == 1:
                split_sen += '\u3000'

        splited.append(split_sen)
    dat['splited'] = pd.DataFrame(splited)

    return dat

def display(dataframe, start=0, stop=None):
    for i in range(len(dataframe.iloc[start:stop,0])):
        print('original sentence:  ' + dataframe.iloc[i, 0])
        print('splitted sentence:  ' + dataframe.iloc[i, 1])
        print()


def display_with_correct(dataframe, correct, start=0, stop=None):
    for i in range(len(dataframe.iloc[start:stop,0])):
        print('original sentence:  ' + dataframe.iloc[i, 0])
        print('splitted sentence:  ' + dataframe.iloc[i, 1])
        print('actually sentence:  ' + correct.iloc[i, 0])
        print()

In [13]:
input_sentences=[
    '生日快樂',
    '我的名字是',
    '今天的天氣很不可思議',
    '台北市國稅局南港稽征處定三月十一日上午九時卅分起至下午四時在活動中心一樓設服務台協助同仁處理綜合所得稅結算申報事宜'
]

In [176]:
input_sentences = list(xtrain.iloc[:10,0])

In [14]:
token_sentences = tokenizer.texts_to_sequences(input_sentences)
token_sentences = pad_sequences(token_sentences, padding='post', maxlen=sentlen)

text_predictions = segcl.predict(token_sentences)
text_predictions = tf.nn.softmax(text_predictions)
text_predictions = np.argmax(text_predictions, axis=-1)

org_data = splitter(input_sentences, text_predictions)

In [ ]:
display_with_correct(org_data, xdf)

In [15]:
display(org_data)

original sentence:  生日快樂
splitted sentence:  生日　快樂　

original sentence:  我的名字是
splitted sentence:  我　的　名字　是　

original sentence:  今天的天氣很不可思議
splitted sentence:  今天　的　天氣　很　不可思議　

original sentence:  台北市國稅局南港稽征處定三月十一日上午九時卅分起至下午四時在活動中心一樓設服務台協助同仁處理綜合所得稅結算申報事宜
splitted sentence:  台北市　國稅局　南港　稽征處　定　三月　十一日　上午　九時　卅分　起　至　下午　四時　在　活動　中心　一樓　設　服務台　協助　同仁　處理　綜合　所　得稅　結算　申報　事宜　

